# Análise de Correspondência Múltipla (MCA) - Adaptação Estudantes

## Contexto

Faremos um estudo exploratório por meio da técnica não supervisionada MCA sobre a adaptação de estudantes ao ensino online. 

**Definição de MCA:** possui o objetivo de analisar a associação entre mais de duas variáveis categóricas. Somente as variáveis que apresentam associação estatisticamente significativa participam da MCA.

Realizamos a leitura de um artigo e extraímos os dados de uma plataforma (Kaggle). Embora o estudo original tenha sido feito em um contexto supervisionado, nosso objetivo é observar como as categorias das variáveis categóricas se associam. Abaixo, escrevemos um resumo desse artigo que pode ser encontrado no Google Acadêmico.

* **Suzan et al.(2021) Students' Adaptability Level Prediction in Online Education using Machine Learning Approaches - DOI: 10.1109/ICCCNT51525.2021.9579741**

O artigo "Students' Adaptability Level Prediction in Online Education using Machine Learning Approaches" aborda a previsão do nível de adaptabilidade dos estudantes em ambientes de educação online utilizando abordagens de aprendizado de máquina. Com o crescimento da educação à distância, tornou-se essencial compreender e prever como os estudantes se adaptam a este novo formato, visando melhorar a experiência de aprendizagem e os resultados acadêmicos.

O estudo utiliza diferentes algoritmos de aprendizado de máquina, como Decision Trees, Random Forest, e Support Vector Machines, para prever a adaptabilidade com base em características comportamentais e demográficas dos estudantes. Além disso, o artigo analisa a precisão desses modelos e identifica os fatores mais relevantes que influenciam a adaptabilidade.

Os resultados mostram que as técnicas de aprendizado de máquina podem oferecer previsões confiáveis, ajudando instituições educacionais a identificar estudantes que podem precisar de suporte adicional. Dessa forma, contribui-se para o desenvolvimento de estratégias personalizadas de ensino que promovem uma experiência de aprendizagem mais inclusiva e eficaz.

## Importação de bibliotecas e pacotes

In [1]:
# Importando os pacotes necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import prince
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'

from scipy.stats import chi2_contingency

## Leitura dos dados

In [2]:
# Importando o banco de dados
dados_estudantes = pd.read_csv("estudantes_adapta.csv")

**Fonte:** adaptado de https://www.kaggle.com/datasets/mdmahmudulhasansuzan/students-adaptability-level-in-online-education

**Obs.:**  O arquivo original apresenta mais colunas (14 no total). Porém, iremos utilizar apenas algumas para podermos aplicar uma Análise de Correspondência Múltipla de forma simples.

## Análise descritiva

Nessa etapa, iremos estudar os dados através de resumos, a fim de observamos padrões e tendências.

In [3]:
# Visualização das 10 primeiras linhas 
dados_estudantes.head(10)

,Education,Institution,Financial,Internet,Adaptivity
0,University,Non Government,Mid,Wifi,Moderate
1,University,Non Government,Mid,Mobile Data,Moderate
2,College,Government,Mid,Wifi,Moderate
3,School,Non Government,Mid,Mobile Data,Moderate
4,School,Non Government,Poor,Mobile Data,Low
5,School,Non Government,Poor,Mobile Data,Low
6,School,Non Government,Mid,Wifi,Low
7,School,Non Government,Mid,Wifi,Moderate
8,College,Government,Mid,Wifi,Low
9,School,Non Government,Mid,Mobile Data,Moderate


In [4]:
# Dimensão dos dados
dados_estudantes.shape

(1205, 5)

In [5]:
# Tipos de cada variável
dados_estudantes.dtypes

Education      object
Institution    object
Financial      object
Internet       object
Adaptivity     object
dtype: object

Podemos observar que temos 1205 observações e 5 variáveis, sendo todas elas categóricas.

In [6]:
# Verificando a presença de nulos
dados_estudantes.isnull().sum()

Education      0
Institution    0
Financial      0
Internet       0
Adaptivity     0
dtype: int64

Nenhuma coluna apresenta valores nulos, ou seja, temos um conjunto de dados totalmente preenchido, não precisando de nenhum tratamento.

Agora, iremos analisar cada variável a fim de verificar quantas categorias cada uma possui e a quantidade de cada uma delas.

In [7]:
# Nível educacional dos estudantes  
dados_estudantes['Education'].value_counts()

Education
School        530
University    456
College       219
Name: count, dtype: int64

Temos 3 tipos de classificação em relação ao nível educacional dos estudantes: escolar, faculdade e universidade. A maioria pertence à fase escolar.

In [8]:
# Tipo de cada instituição
dados_estudantes['Institution'].value_counts()

Institution
Non Government    823
Government        382
Name: count, dtype: int64

Temos 2 tipos de instituição: governamental e não governamental, com a maioria pertencente à não governamental.

In [9]:
# Nível financeiro da família do estudante
dados_estudantes['Financial'].value_counts()

Financial
Mid     878
Poor    242
Rich     85
Name: count, dtype: int64

Nessa variável, o nível financeiro é dividido em três categorias: pobre, médio e rico, com a maioria dos estudantes classificados como médios e, a minoria, como ricos.

In [10]:
# Tipo de internet
dados_estudantes['Internet'].value_counts()

Internet
Mobile Data    695
Wifi           510
Name: count, dtype: int64

A internet foi dividida em móvel (maioria) e Wifi.

In [11]:
# Adaptação dos estudantes ao modo online
dados_estudantes['Adaptivity'].value_counts()

Adaptivity
Moderate    625
Low         480
High        100
Name: count, dtype: int64

A adaptação dos estudantes ao ensino online foi dividida em 3 categorias, sendo elas: baixa, moderada e alta. Nessa primeira análise, mostrou-se que a maioria dos estudantes se encaixam em um nível moderado, com poucos sendo classificados no nível alto.

## Análise diagnóstica

Nesse tipo de análise, os dados são investigados com o intuito de encontrarmos relações de causa e efeito. O foco está em descobrir razões por trás dos dados.

No nosso caso, queremos investigar como essas variáveis se associam. Para isso, nosso ponto de partida, para uma Análise de Correspondência, serão as tabelas de contingência entre pares. 

Uma tabela de contingência nada mais é do que uma tabela que contém as frequências absolutas observadas para cada par de categorias das variáveis. Trata-se de uma tabela de classificação cruzada.

Como as combinações são muitas, iremos gerar essas tabelas em relação apenas à variável "Adaptativity", já que é o foco do estudo em questão. O objetivo é gerar essas tabelas para que possam ser usadas na função qui-quadrado que compõe o teste qui-quadrado.

O teste qui-quadrado avalia se há associação estatisticamente significativa entre as variáveis. Logo, as hipóteses são:

 * $H_0$ : as variáveis se associam de forma aleatória (não há associação significativa).

 * $H_1$ : há associação significativa entre as variáveis (forma não aleatória).

Os graus de liberdade são dados pela seguinte fórmula:

<div align="center">
(I - 1) * (J - 1),
</div>

em que I é a quantidade de categorias em linha e J, a quantidade de categorias em coluna.

Para validação, definimos um nível de significância de 5%.

In [12]:
# Adaptação X Educação
tab_cont_edu = pd.crosstab(dados_estudantes["Adaptivity"], dados_estudantes["Education"])
tab_cont_edu

Education,College,School,University
Adaptivity,,,
High,3,47,50
Low,120,182,178
Moderate,96,301,228


In [13]:
# Adaptação X Instituição
tab_cont_inst = pd.crosstab(dados_estudantes["Adaptivity"], dados_estudantes["Institution"])
tab_cont_inst

Institution,Government,Non Government
Adaptivity,,
High,20,80
Low,234,246
Moderate,128,497


In [14]:
# Adaptação X Nível Financeiro
tab_cont_fin = pd.crosstab(dados_estudantes["Adaptivity"], dados_estudantes["Financial"])
tab_cont_fin

Financial,Mid,Poor,Rich
Adaptivity,,,
High,36,22,42
Low,341,129,10
Moderate,501,91,33


In [15]:
# Adaptação X Internet
tab_cont_inter = pd.crosstab(dados_estudantes["Adaptivity"], dados_estudantes["Internet"])
tab_cont_inter

Internet,Mobile Data,Wifi
Adaptivity,,
High,36,64
Low,288,192
Moderate,371,254


In [16]:
# Aplicando o teste qui-quadrado para Adaptação X Educação
teste_qui2_edu = chi2_contingency(tab_cont_edu)

print("Adaptivity x Education")
print("")
print(f"Estatística qui²: {round(teste_qui2_edu[0], 2)}")
print(f"Graus de liberdade: {teste_qui2_edu[2]}")
print(f"p-valor da estatística: {round(teste_qui2_edu[1], 4)}")

Adaptivity x Education

Estatística qui²: 38.69
Graus de liberdade: 4
p-valor da estatística: 0.0


In [17]:
# Aplicando o teste qui-quadrado para Adaptação X Instituição
teste_qui2_inst = chi2_contingency(tab_cont_inst)

print("Adaptivity x Institution")
print("")
print(f"Estatística qui²: {round(teste_qui2_inst[0], 2)}")
print(f"Graus de liberdade: {teste_qui2_inst[2]}")
print(f"p-valor da estatística: {round(teste_qui2_inst[1], 4)}")

Adaptivity x Institution

Estatística qui²: 107.11
Graus de liberdade: 2
p-valor da estatística: 0.0


In [18]:
# Aplicando o teste qui-quadrado para Adaptação X Nível Financeiro
teste_qui2_fin = chi2_contingency(tab_cont_fin)

print("Adaptivity x Financial")
print("")
print(f"Estatística qui²: {round(teste_qui2_fin[0], 2)}")
print(f"Graus de liberdade: {teste_qui2_fin[2]}")
print(f"p-valor da estatística: {round(teste_qui2_fin[1], 4)}")

Adaptivity x Financial

Estatística qui²: 236.86
Graus de liberdade: 4
p-valor da estatística: 0.0


In [19]:
# Aplicando o teste qui-quadrado para Adaptação X Internet
teste_qui2_inter = chi2_contingency(tab_cont_inter)

print("Adaptivity x Financial")
print("")
print(f"Estatística qui²: {round(teste_qui2_inter[0], 2)}")
print(f"Graus de liberdade: {teste_qui2_inter[2]}")
print(f"p-valor da estatística: {round(teste_qui2_inter[1], 4)}")

Adaptivity x Financial

Estatística qui²: 21.04
Graus de liberdade: 2
p-valor da estatística: 0.0


Em todos os testes realizados entre os pares de variáveis, observamos que o p-valor é menor do que o nível de significância de 5% (0.0 < 0.05). Portanto, rejeitamos a hipótese nula e concluímos que as associações são estatisticamente significativas. Dessa maneira, podemos incluir todas as variáveis na Análise de Correspondência Múltipla.

## Análise de Correspondência Múltipla

Iremos parametrizar a MCA para três dimensões, já que o nosso objetivo é criar um mapa perceptual 3D. Esse mapa é um gráfico de pontos, no qual cada ponto será uma categoria das variáveis.

In [20]:
# Elaborando a MCA para 3 dimensões (n_components=3) com o banco de dados 
mca_estudantes = prince.MCA(n_components=3).fit(dados_estudantes)

In [21]:
# Resumo das informações da MCA
quant_dim = mca_estudantes.J_ - mca_estudantes.K_

print(f"Quantidade total de categorias: {mca_estudantes.J_}")
print(f"Quantidade de variáveis: {mca_estudantes.K_}")
print(f"Quantidade de dimensões: {quant_dim}")

Quantidade total de categorias: 13
Quantidade de variáveis: 5
Quantidade de dimensões: 8


Como podemos notar, a quantidade de dimensões para esse conjunto de dados são oito. Contudo, nós somos capazes de visualizar em até três dimensões.

Para construirmos o mapa perceptual, precisamos calcular os autovalores, no qual cada autovalor refere-se a uma dimensão no mapa. 

In [22]:
# Obtendo os autovalores (eigenvalues)
tab_autovalores = mca_estudantes.eigenvalues_summary
tab_autovalores

,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,0.321,20.06%,20.06%
1,0.308,19.24%,39.30%
2,0.258,16.10%,55.40%


Quando obtemos os autovalores, eles sempre aparecem do maior para o menor valor. No caso, considera-se três dimensões, as quais correspondem à 55.4% da variância (inércia) dos dados. Lembrando que o zero é o eixo "x", o um, o eixo "y" e o dois, o eixo "z".

Com base nos autovalores obtidos, encontra-se o percentual de inércia principal total de cada dimensão, que é a soma de todos os autovalores (todas as dimensões existentes).

In [23]:
# Calculando a inércia principal total
inercia_principal_total = mca_estudantes.total_inertia_
inercia_principal_total

1.5999999999999586

Se fossem somados todos os autovalores, a inércia seria 1.599. 

Quando plotamos um mapa perceptual de uma Análise de Correspondência Múltipla (no caso, três dimensões), é recomendado que utilizemos somente dimensões que tenham um autovalor maior do que a inércia total média, ou seja, a média da inércia principal total por dimensão.

In [24]:
# Média da inércia principal total por dimensão
inercia_total_media = mca_estudantes.total_inertia_/quant_dim
inercia_total_media

0.19999999999999482

Neste caso, as três dimensões extraídas possuem autovalores maiores do que 0.199. Se acontecesse de tivermos, por exemplo, no eixo "z" um valor menor do que 0.199, significa que essa dimensão tem pouca informação para ser adicionada na análise. Logo, retiraríamos esse eixo do nosso estudo, já que não ajudaria na análise de proximidade do mapa.

A seguir, calculamos as coordenadas das categorias das variáveis e das observações. Aqui, podemos utilizar a Matriz de Burt, que são todas as categorias em linha e todas as categorias em coluna. Assim, essa matriz mostra a quantidade de cada par de categorias. Essa matriz gera as coordenadas principais com base na multiplicação entre a matriz biária Z transposta e a matriz binária Z:

<div align="center">
B = Z' * Z,
</div>

no qual a matriz binária é obtida pela transformação das variáveis qualitativas em variáveis binárias, isto é, em valores 0 ou 1 ("dummies").

Dessa forma, combinamos em uma única matriz o cruzamento de todos os pares de variáveis e suas categorias, obtendo uma matriz que contém as frequências absolutas observadas para todos os cruzamentos.

Ao considerar a Matriz de Burt como uma tabela de contingência, é possível realizar uma Análise de Correspondência e obter as coordenadas das categorias das variáveis.

In [25]:
# Obtendo as coordenadas principais das categorias das variáveis
# Como default, a função "column_coordinates" traz as coordenadas de Burt
coord_burt = mca_estudantes.column_coordinates(dados_estudantes)
coord_burt

,0,1,2
Education_College,-1.266116,0.559165,-0.011469
Education_School,0.476726,-0.675321,0.099547
Education_University,0.053979,0.516366,-0.110193
Institution_Government,-0.846907,0.380226,0.692520
Institution_Non Government,0.393097,-0.176484,-0.321437
Financial_Mid,-0.262792,0.125889,-0.393927
Financial_Poor,0.245959,-1.023834,0.979460
Financial_Rich,2.014228,1.614552,1.280454
Internet_Mobile Data,-0.057200,-0.631284,0.212550
Internet_Wifi,0.077949,0.860279,-0.289652


Caso queiramos as coordenadas padrão, basta dividirmos "coord_burt" pela raíz dos autovalores.

In [26]:
# Obtendo as coordenadas-padrão das categorias das variáveis
coord_padrao = coord_burt/np.sqrt(mca_estudantes.eigenvalues_)
coord_padrao

,0,1,2
Education_College,-2.234816,1.007914,-0.022596
Education_School,0.841467,-1.217289,0.196122
Education_University,0.095278,0.930767,-0.217097
Institution_Government,-1.494873,0.685371,1.364369
Institution_Non Government,0.693853,-0.318119,-0.633279
Financial_Mid,-0.463853,0.226920,-0.776095
Financial_Poor,0.434140,-1.845496,1.929683
Financial_Rich,3.555306,2.910286,2.522688
Internet_Mobile Data,-0.100964,-1.137912,0.418756
Internet_Wifi,0.137588,1.550684,-0.570658


Sendo a matriz binária Z responsável por gerar as coordenadas padrão e considerando-a como a tabela de contingência na Análise de Correspondência, é exequível obter a inércia principal das dimensões, autovalores e, portanto, as coordenadas no mapa perceptual dessa matriz.

In [27]:
# Obtendo as coordenadas das observações do banco de dados
# Na função "row_coordinates", as coordenadas das observações vêm das coordenadas-padrão
coord_obs = mca_estudantes.row_coordinates(dados_estudantes)
coord_obs

,0,1,2
0,0.164470,0.410168,-0.719597
1,0.116760,-0.127551,-0.521715
2,-0.739294,0.626296,-0.281168
3,0.265998,-0.557162,-0.439071
4,0.135029,-0.914878,0.640151
...,...,...,...
1200,-0.612116,0.482365,-0.142631
1201,-0.301548,0.425598,-0.680697
1202,0.265998,-0.557162,-0.439071
1203,-0.612116,0.482365,-0.142631


Feito isso, conseguimos plotar o mapa perceptual utilizando as coordenadas padrão e, consequentemente, visualizar as categorias que estão mais associadas.

## Mapa Perceptual

In [28]:
# Plotando o mapa perceptual (coordenadas-padrão)

# Primeiro passo: gerar um DataFrame detalhado
coord = coord_padrao.reset_index() # A primeira coluna será o nome das categorias

var_coord = pd.Series(coord['index'].str.split('_', expand=True).iloc[:,0])

# Extraindo o nome das categorias (sem as variáveis)
nome_categ=[]
for col in dados_estudantes:
    nome_categ.append(dados_estudantes[col].sort_values(ascending=True).unique())
    categorias = pd.DataFrame(nome_categ).stack().reset_index()

coord_df_mca = pd.DataFrame({'categoria': coord['index'],
                             'obs_x': coord[0],
                             'obs_y': coord[1],
                             'obs_z': coord[2],
                             'variavel': var_coord,
                             'categoria_id': categorias[0]})

# Segundo passo: gerar o gráfico de pontos
fig = px.scatter_3d(coord_df_mca, 
                    x='obs_x', 
                    y='obs_y', 
                    z='obs_z',
                    color='variavel',
                    text=coord_df_mca.categoria_id)
fig.show()

## Conclusão

Portanto, estudantes que reportaram alta adaptação ao ensino online, possuem um nível financeiro elevado. Também notamos que alunos que possuem baixa adaptabilidade estudam em instituições governamentais. Estudantes que frequentam a universidade estão mais associados ao uso do Wifi e com um nível financeiro mediano. Os alunos que declararam ter uma adaptabilidade mediana ao ensino online, frequentam instituições não governamentais. Já os estudantes de condição financeira baixa, não apresentam associações evidentes com outras categorias.